In [603]:
# imports
import pandas as pd
import numpy as np
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize as nltk_tokenize
from collections import Counter as collections_counter
import string
from sklearn.feature_extraction.text import CountVectorizer as sk_feature_extract_text_CV
from random import randint as random_randint

Spam/Ham SMS data was obtained from the Kaggle project <br>
https://www.kaggle.com/sid321axn/sms-spam-classifier-naive-bayes-ml-algo.<br>
The data was loaded into a dataframe and the messages were extracted for cleaning.

In [604]:
# load data

_data = pd.read_csv('./spam.csv')

In [605]:
# show head

_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [606]:
def preprocess(df):

    """
        Modifies data dataframe in place.
            - to lower
            - removes stop words     
            - removes punctuation       
        Parameters:
            df (dataframe): dataframe, column 1 = labels, column 2 = message text
        Returns:
            (None)
    """

    _stopwords = set(nltk_stopwords.words('english'))

    for _m in df.iterrows():
        _m[1][1] = _m[1][1].lower()
        _m[1][1] = _m[1][1].translate(str.maketrans("","", string.punctuation))
        _m[1][1] = nltk_tokenize(_m[1][1])
        _m[1][1] = [_w for _w in _m[1][1] if not _w in _stopwords]
        _m[1][1] = ' '.join(_m[1][1])




In [607]:
# preprocess

preprocess(_data)

The data needs to be separated by spam/ham category for term/document and term frequency analysis.

In [608]:
# separate based on label

_spam = _data[_data['Category'] == 'spam']
_ham = _data[_data['Category'] == 'ham']

We implemented our bag of words by creating a term/document (sparse) matrix and a term frequency (condensed) matrix in order to conduct Baysian analysis. <br>
We used a method to create term/document matrices from the following site: https://www.kaggle.com/sid321axn/sms-spam-classifier-naive-bayes-ml-algo <br>
This method invoved using the Count Vectorizer class from sklearn to create a list of counts of words found in the data as well as a list of the words themselves. <br>
The rows of the dataframe will represent word counts for a given word. <br>
The columns will represent the words found in the messages.

In [609]:
def generate_tdm(df):
    
    """
        generates a term document matrix from input data
        Parameters:
            df (DataFrame): dataframe with column 1 = labels, column 2 = messages
        Returns:
            DataFrame: a term document matrix
    """
    
    _count_vectorizer = sk_feature_extract_text_CV()
    _count_vectorizer.fit(df.iloc[:,1])
    _features = _count_vectorizer.get_feature_names_out()
    _counts = _count_vectorizer.transform(df.iloc[:,1]).toarray()
    return pd.DataFrame(_counts, columns=_features)


In [610]:
# generate tdm for spam and ham

_tdm = generate_tdm(_data)
_spam_tdm = generate_tdm(_spam)
_ham_tdm = generate_tdm(_ham)

The frequency matrices were dataframes with one row for each word found in the dataset an two columns, <br> 
one for the number of times the word appeard and the other for the word's frequency among the messages.

In [611]:
def generate_tfm(df):
    
    """
        generates a term frequency matrix from a term document matrix
        Parameters:
            df (DataFrame): should be a term document matrix
        Returns:
            DataFrame: a term frequency matrix
    """
    _counts = []
    _probability = []
    for _label, _data in df.iteritems():
        _sum = _data.sum()
        _counts.append(_sum)
    _sum_of_all_words = sum(_counts)
    for _c in _counts:
        _probability.append((_c + 1) / (_sum_of_all_words + df.shape[1] + 1)) # shape[1] is the number of unique words in the class
    _out_data = {
        'word': df.columns,
        'count': _counts,
        'probability': _probability
    }
    return pd.DataFrame().from_dict(_out_data)


In [612]:
# generated term frequency matrix

_spam_tfm = generate_tfm(_spam_tdm)
_ham_tfm = generate_tfm(_ham_tdm)

In [614]:
# look at spam tfm

_spam_tfm.sort_values(by=['probability'], ascending=False)

,word,count,probability
972,call,344,0.023339
1367,free,216,0.014680
2607,txt,150,0.010215
2644,ur,144,0.009809
1825,mobile,123,0.008389
...,...,...,...
1205,disaster,1,0.000135
1203,dirty,1,0.000135
1202,dirtiest,1,0.000135
1200,dining,1,0.000135


In [615]:
# look at ham tfm

_ham_tfm.sort_values(by=['probability'], ascending=False)

,word,count,probability
3194,im,451,0.010015
2645,get,303,0.006735
3889,ltgt,276,0.006137
4574,ok,273,0.006071
1928,dont,265,0.005894
...,...,...,...
3149,hwkeep,1,0.000044
3148,hwd,1,0.000044
3145,hut,1,0.000044
3144,hustle,1,0.000044


We needed the probabilty for each class; the probability that a given message was spam or ham.

In [616]:
def calculate_class_probability(df):

    """
        spam = #_spam/#_messages; ham = #_ham/#_messages
        Parameters:
            df (DataFrame): the data set; can be pre or post processed
        Returns: 
            (dict): {spam: (float), ham: (float)}
    """

    _count = df.shape[0]
    _p = {
        'spam': df[df['Category'] == 'spam'].count()[0] / _count,
        'ham': df[df['Category'] == 'ham'].count()[0] / _count
    }
    return _p

In [617]:
# calculate probabilites for each class

_p_classes = calculate_class_probability(_data)
print(_p_classes)

{'spam': 0.13406317300789664, 'ham': 0.8659368269921034}


In [626]:
def calculate_probability(message, p_class, class_tfm):

    """
        calculates the probability that a given message belongs to 
            the class of the class_tfm
        Parameters:
            message (string): a single message from the data set
            p_class (float): probability of class in data set
            class_tfm (DataFrame): message probability calculated
                for this class's tfm
            data_tfm (DataFrame): data set's term frequency matrix
        Returns:
            float: the probability that the message belongs to the 
                label of the supplied class_tfm
    """

    _words = message.split()
    _words_in_class = class_tfm['count'].sum()
    _p_word_not_in_class = 1 / (1 + _words_in_class + class_tfm.shape[0])  # shape[0] is the number of unique words in the class
    _p = 1
    
    for _w in _words:
        
        # word not in class_tfm
        _row = class_tfm[class_tfm['word'] == _w]
        if _row.empty:
            _p_word = _p_word_not_in_class

        # word in class_tfm
        else:
            _p_word = _row['probability'].iloc[0]

        # product
        _p *= _p_word

    return _p * p_class


In [627]:
# make predictions

_predictions = []
for _row in _data.iterrows():
    _p_spam = calculate_probability(_row[1][1], _p_classes['spam'], _spam_tfm)
    _p_ham = calculate_probability(_row[1][1], _p_classes['ham'], _ham_tfm)
    _prediction = 'spam'
    if _p_ham > _p_spam:
        _prediction = 'ham'
    _predictions.append(_prediction)

In [628]:
# compare predictions

_sum = 0
for _i,_row in enumerate(_data.iterrows()):
    if _row[1][0] != _predictions[_i]:
        continue
    _sum += 1
_score = _sum / _data.shape[0]
print(_score)

0.9761306532663316
